在深度学习任务中，数据加载和处理是至关重要的一环。

PyTorch 提供了强大的数据加载和处理工具，主要包括：

* torch.utils.data.Dataset：数据集的抽象类，需要自定义并实现 __len__（数据集大小）和 __getitem__（按索引获取样本）。

* torch.utils.data.TensorDataset：基于张量的数据集，适合处理数据-标签对，直接支持批处理和迭代。

* torch.utils.data.DataLoader：封装 Dataset 的迭代器，提供批处理、数据打乱、多线程加载等功能，便于数据输入模型训练。

* torchvision.datasets.ImageFolder：从文件夹加载图像数据，每个子文件夹代表一个类别，适用于图像分类任务。

PyTorch 通过 torchvision.datasets 模块提供了许多常用的数据集，例如：

* MNIST：手写数字图像数据集，用于图像分类任务。

* CIFAR：包含 10 个类别、60000 张 32x32 的彩色图像数据集，用于图像分类任务。

* COCO：通用物体检测、分割、关键点检测数据集，包含超过 330k 个图像和 2.5M 个目标实例的大规模数据集。

* ImageNet：包含超过 1400 万张图像，用于图像分类和物体检测等任务。

* STL-10：包含 100k 张 96x96 的彩色图像数据集，用于图像分类任务。

* Cityscapes：包含 5000 张精细注释的城市街道场景图像，用于语义分割任务。

* SQUAD：用于机器阅读理解任务的数据集。

  
以上数据集可以通过 torchvision.datasets 模块中的函数进行加载，也可以通过自定义的方式加载其他数据集。

In [4]:
# Dataset 是 PyTorch 中用于数据集抽象的类。
import torch
from torch.utils.data import Dataset

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        # 数据初始化
        self.data = data
        self.labels = labels

    def __len__(self):
        # 返回数据集的大小
        return len(self.data)

    def __getitem__(self, idx):
        # 按索引返回数据和标签
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label


# 生成示例数据
data = torch.randn(100, 5)  # 100 个样本，每个样本有 5 个特征
labels = torch.randint(0, 2, (100,))  # 100 个标签，取值为 0 或 1

# 实例化数据集
dataset = MyDataset(data, labels)

# 测试数据
print("数据集大小： ", len(data))  
print("第0个样本： ", dataset[0])
print("第0个样本的标签：", labels[0])

数据集大小：  100
第0个样本：  (tensor([-0.7846, -2.0115,  1.1344, -0.1920,  0.6623]), tensor(0))
第0个样本的标签： tensor(0)


In [17]:
# 实例化DataLoader
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)

#遍历dataloader
for batch_idx, (batch_data, batch_labels) in enumerate(dataloader):
    print(f"批次 {batch_idx + 1}")
    print("数据:", batch_data)
    print("标签:", batch_labels)
    if batch_idx == 2:  # 仅显示前 3 个批次
        break

批次 1
数据: tensor([[ 0.5918, -1.6066,  0.8331,  1.0275,  0.5360],
        [-0.9193, -0.3057,  0.4550,  0.2188, -1.6957],
        [ 0.4441, -1.3283, -0.3794,  1.9294,  0.0343],
        [ 0.6481,  0.4386, -0.6177, -0.6980,  1.0279],
        [-1.4598,  0.8592,  0.5620, -0.6764,  0.0909],
        [-1.2711,  0.0726,  1.2691, -0.2820,  0.8002],
        [ 0.3931,  0.4277,  1.2196, -1.0251,  0.6803],
        [ 0.7258, -1.3867, -0.2223,  0.6812,  2.0169],
        [ 0.4721, -1.2003,  0.8331,  2.2153,  1.0302],
        [ 1.3101,  0.6708, -1.1799, -0.1685,  1.1165]])
标签: tensor([0, 1, 0, 1, 1, 1, 1, 0, 0, 1])
批次 2
数据: tensor([[ 0.2007, -0.3181, -1.2759,  0.5157,  0.2088],
        [-0.6246,  0.1202, -0.7655, -0.4429, -0.5134],
        [-0.7184, -0.1589, -0.0243,  0.8474, -0.5594],
        [ 0.1148,  0.9726, -0.5741,  0.2793,  0.2531],
        [-0.0650, -0.6639, -0.2864, -0.3152,  1.7142],
        [-0.4697, -0.3783, -0.3046, -0.0920, -0.1208],
        [ 0.8554, -0.9507,  0.0170,  0.2815,  1.0621],
   

In [19]:
# Dataset 与 DataLoader 的自定义应用
# 以下是一个将 CSV 文件 作为数据源，并通过自定义 Dataset 和 DataLoader 读取数据

In [21]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [25]:
# 自定义CSV数据集
class CSVDataset(Dataset):
    def __init__(self, file_path):
        # 读取CSV文件
        self.data = pd.read_csv(file_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 使用 .iloc 明确基于位置索引
        row = self.data.iloc[idx]
        # 将特征和标签分开
        features = torch.tensor(row.iloc[:-1].to_numpy(), dtype=torch.float32)
        label = torch.tensor(row[:-1], dtype=torch.float32)
        return features, label

# 实例化数据集和DataLoader
dataset = CSVDataset("test_pytorch_data.csv")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# 遍历 DataLoader
for features, label in dataloader:
    print("特征:", features)
    print("标签:", label)
    break

特征: tensor([[ 2.1000, -3.3000,  0.0000],
        [-0.8000,  0.0000,  0.3000],
        [ 0.5000, -1.2000,  3.3000],
        [ 0.9000, -0.5000, -1.8000]])
标签: tensor([[ 2.1000, -3.3000,  0.0000],
        [-0.8000,  0.0000,  0.3000],
        [ 0.5000, -1.2000,  3.3000],
        [ 0.9000, -0.5000, -1.8000]])


C:\Users\kai\AppData\Local\Temp\ipykernel_12464\648897167.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = torch.tensor(row[:-1], dtype=torch.float32)
